**Text Mining for AI - Group1 Project** \
First step is acquiring dataset. \
Second step is processing data, for now : \
- we remove duplicates review 
- lower case everything 
- removes rows where the 'app_name' column is null or an empty string

In [4]:
import pandas as pd
import re

df = pd.read_csv('steam_reviews.csv')
#remove dublicate reviews
df = df.drop_duplicates(subset='review_text', keep='first')
#all in lowercase 
df['review_text'] = df['review_text'].str.lower()
df = df[df['review_text'].notna() & (df['review_text'] != '')]
df = df[df['app_name'].notna() & (df['app_name'] != '')]
df = df[df['language'] == 'english']

df = df.drop(columns=['timestamp_created', 'timestamp_updated', 'comment_count', "steam_china_location", "language"])

#print(df.head(10))

KeyboardInterrupt: 

Saving the data as new csv file. 

In [18]:
#df.to_csv('/Volumes/PotatoExpan/Text_Mining_Project/processed_steam_reviews.csv')
# Specify a separator and ensure headers are included
#df.to_csv('/Volumes/PotatoExpan/Text_Mining_Project/secondtry.csv', sep=';', index=True, header=True)
df.to_csv('/Volumes/PotatoExpan/Text_Mining_Project/only_english.csv', sep=';', index=False, header=True, line_terminator='\n')

